In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM, Bidirectional, Dropout, Input, Layer, GlobalMaxPooling1D, LayerNormalization
from tensorflow.keras.models import Model
from transformers import BertTokenizer, TFBertModel
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [ ]:
df = pd.read_excel("/content/PERC_mendelly.xlsx")

In [ ]:
def map_emotions(emotion):
    positive = ['courage', 'joy', 'love', 'peace', 'surprise']
    negative = ['anger', 'fear', 'hate', 'sad']
    return 'positive' if emotion in positive else 'negative' if emotion in negative else 'neutral'

df['Sentiment'] = df['Emotion'].apply(map_emotions)

In [ ]:
# Tokenization
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
def tokenize_poems(text, max_len=128):
    return tokenizer(text, max_length=max_len, padding='max_length', truncation=True, return_tensors='tf')
X = list(df['Poem'].values)
X_tokenized = [tokenize_poems(poem) for poem in X]
X_input_ids = np.array([x['input_ids'][0].numpy() for x in X_tokenized])
X_attention_masks = np.array([x['attention_mask'][0].numpy() for x in X_tokenized])

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
# Encode sentiments
label_encoder = LabelEncoder()
df['Sentiment_encoded'] = label_encoder.fit_transform(df['Sentiment'])
y = df['Sentiment_encoded'].values

In [ ]:
y_train = np.argmax(to_categorical(y), axis=1)

In [ ]:
X_train_input_ids, X_val_input_ids, X_train_attention_masks, X_val_attention_masks, y_train, y_val = train_test_split(
    X_input_ids, X_attention_masks, y_train, test_size=0.2, stratify=y_train, random_state=42)

In [ ]:
# Load BERT model
bert_model = TFBertModel.from_pretrained('bert-base-uncased')
for layer in bert_model.layers:
    layer.trainable = True

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [ ]:
# Custom BERT layer
class BertLayer(Layer):
    def __init__(self, **kwargs):
        super(BertLayer, self).__init__(**kwargs)
        self.bert = bert_model

    def call(self, inputs):
        input_ids, attention_mask = inputs
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        return outputs.last_hidden_state

In [ ]:
# Model architecture
max_len = 128
input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
attention_masks = Input(shape=(max_len,), dtype=tf.int32, name="attention_masks")

bert_embeddings = BertLayer()([input_ids, attention_masks])
bert_embeddings = LayerNormalization()(bert_embeddings)

In [ ]:
# Simplified architecture with more dropout
x = Bidirectional(LSTM(64, return_sequences=True))(bert_embeddings)
x = GlobalMaxPooling1D()(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.3)(x)

output_layer = Dense(3, activation='softmax')(x)

model = Model(inputs=[input_ids, attention_masks], outputs=output_layer)

In [ ]:
# Compile
optimizer = Adam(learning_rate=3e-5, clipnorm=1.0)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Compute class weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = dict(enumerate(class_weights))

In [ ]:
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6)

In [ ]:
# Train the model
history = model.fit([X_train_input_ids, X_train_attention_masks], y_train,
                    validation_data=([X_test_input_ids, X_test_attention_masks], y_test),
                    epochs=15,
                    batch_size=16,
                    class_weight=class_weights_dict,
                    callbacks=[early_stopping, reduce_lr],
                    verbose=1)

Epoch 1/15
36/36 ━━━━━━━━━━━━━━━━━━━━ 25s 280ms/step - accuracy: 0.2641 - loss: 1.2685 - val_accuracy: 0.5000 - val_loss: 0.8605 - learning_rate: 3.0000e-05
Epoch 2/15
36/36 ━━━━━━━━━━━━━━━━━━━━ 7s 198ms/step - accuracy: 0.5006 - loss: 0.8719 - val_accuracy: 0.6042 - val_loss: 0.7347 - learning_rate: 3.0000e-05
Epoch 3/15
36/36 ━━━━━━━━━━━━━━━━━━━━ 10s 198ms/step - accuracy: 0.5755 - loss: 0.7706 - val_accuracy: 0.6528 - val_loss: 0.6906 - learning_rate: 3.0000e-05
Epoch 4/15
36/36 ━━━━━━━━━━━━━━━━━━━━ 10s 197ms/step - accuracy: 0.6139 - loss: 0.7192 - val_accuracy: 0.6736 - val_loss: 0.6667 - learning_rate: 3.0000e-05
Epoch 5/15
36/36 ━━━━━━━━━━━━━━━━━━━━ 7s 191ms/step - accuracy: 0.5425 - loss: 0.7365 - val_accuracy: 0.7014 - val_loss: 0.6482 - learning_rate: 3.0000e-05
Epoch 6/15
36/36 ━━━━━━━━━━━━━━━━━━━━ 7s 194ms/step - accuracy: 0.6187 - loss: 0.7025 - val_accuracy: 0.7569 - val_loss: 0.6209 - learning_rate: 3.0000e-05
Epoch 7/15
36/36 ━━━━━━━━━━━━━━━━━━━━ 10s 194ms/step - accura

In [ ]:
# Evaluation
loss, accuracy = model.evaluate([X_test_input_ids, X_test_attention_masks], y_test)
print(f"Test Accuracy: {accuracy:.2f}, Test Loss: {loss:.4f}")

5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 249ms/step - accuracy: 0.9044 - loss: 0.4738
Test Accuracy: 0.89, Test Loss: 0.4762


In [ ]:
sample_poem = "A new dawn, a day of hope and love, embracing challenges, yet rising above."
sample_tokenized = tokenize_poems(sample_poem)
sample_input_ids = sample_tokenized['input_ids'].numpy()
sample_attention_masks = sample_tokenized['attention_mask'].numpy()

In [ ]:
sample_prediction = model.predict([sample_input_ids, sample_attention_masks])
predicted_label = label_encoder.inverse_transform([np.argmax(sample_prediction)])
print(f"Sample Poem: \"{sample_poem}\"")
print(f"Predicted Sentiment: {predicted_label[0]}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Sample Poem: "A new dawn, a day of hope and love, embracing challenges, yet rising above."
Predicted Sentiment: positive
